In [10]:
import src.data as data

data.DataFetcher.main()
data.PreProcess.main(False)

from run_model import ModelPred

model_preds = [
    ModelPred('gru_day' , 'swalast' , 0 , 'gru_day_V0')
]
[md.get_df().deploy() for md in model_preds]

Update Files
Tue Apr  2 11:13:49 2024 : Updated ~ data\DataBase\DB_information\calendar.feather Done! Cost 0.05 Secs
Tue Apr  2 11:13:49 2024 : Updated ~ data\DataBase\DB_information\description.feather Done! Cost 0.09 Secs
Tue Apr  2 11:13:49 2024 : Updated ~ data\DataBase\DB_information\st.feather Done! Cost 0.03 Secs
Tue Apr  2 11:13:51 2024 : Updated ~ data\DataBase\DB_information\industry.feather Done! Cost 2.57 Secs
Tue Apr  2 11:13:56 2024 : Updated ~ data\DataBase\DB_information\concepts.feather Done! Cost 4.68 Secs


24-04-02 11:14:10|MOD:PreProcess  |: if_train is False , Data Processing start!
24-04-02 11:14:10|MOD:PreProcess  |: 3 datas :['y', 'trade_day', 'trade_30m']


Tue Apr  2 11:14:10 2024 : All Updates Done! Cost 21.91 Secs
Tue Apr  2 11:14:10 2024 : y start ...
labels blocks reading ret10_lag DataBase's ...... cost 0.55 secs
labels blocks reading ret20_lag DataBase's ...... cost 0.49 secs
labels blocks merging ...... cost 0.05 secs
models blocks reading risk_exp DataBase's ...... cost 1.93 secs
models blocks merging ...... cost 0.19 secs
y blocks process ...... cost 2.61 secs
y blocks masking ...... cost 0.10 secs
y blocks saving  ...... cost 0.13 secs
y blocks norming ...... cost 0.00 secs
Tue Apr  2 11:14:16 2024 : y finished! Cost 6.05 Seconds
Tue Apr  2 11:14:17 2024 : trade_day start ...
trade blocks reading day DataBase's ...... cost 1.00 secs
trade_day blocks merging ...... cost 0.00 secs
trade_day blocks process ...... cost 0.08 secs
trade_day blocks masking ...... cost 0.11 secs
trade_day blocks saving  ...... cost 0.14 secs
trade_day blocks norming ...... cost 0.00 secs
Tue Apr  2 11:14:18 2024 : trade_day finished! Cost 1.33 Seconds


24-04-02 11:14:25|MOD:PreProcess  |: Data Processing Finished! Cost 14.22 Seconds


... cost 1.03 secs
trade_30m blocks norming ...... cost 0.00 secs
Tue Apr  2 11:14:24 2024 : trade_30m finished! Cost 6.46 Seconds


24-04-02 11:14:30|MOD:PreProcess  |: if_train is True , Data Processing start!
24-04-02 11:14:30|MOD:PreProcess  |: 3 datas :['y', 'trade_day', 'trade_30m']


Tue Apr  2 11:14:30 2024 : y start ...
labels blocks reading ret10_lag DataBase's ...... cost 23.32 secs
labels blocks reading ret20_lag DataBase's ...... cost 20.20 secs
labels blocks merging ...... cost 2.80 secs
models blocks reading risk_exp DataBase's ...... cost 396.61 secs
models blocks merging ...... cost 164.44 secs
y blocks process ...... cost 164.23 secs
y blocks masking ...... cost 1.57 secs
y blocks saving  ...... cost 7.37 secs
y blocks norming ...... cost 0.00 secs
Tue Apr  2 11:27:31 2024 : y finished! Cost 780.74 Seconds
Tue Apr  2 11:27:32 2024 : trade_day start ...
trade blocks reading day DataBase's ...... cost 34.85 secs
trade_day blocks merging ...... cost 0.00 secs
trade_day blocks process ...... cost 3.47 secs
trade_day blocks masking ...... cost 0.78 secs
trade_day blocks saving  ...... cost 5.23 secs
trade_day blocks norming ...... cost 41.30 secs
Tue Apr  2 11:28:58 2024 : trade_day finished! Cost 85.74 Seconds
Tue Apr  2 11:28:58 2024 : trade_30m start ...
t

MemoryError: 